In [1]:
import os
import yaml

import pandas as pd
from itertools import product
import comet_ml

In [2]:
comet_ml.init()

In [3]:
import ultralytics

In [4]:
ultralytics.checks()

Ultralytics YOLOv8.0.112  Python-3.11.3 torch-2.0.1+cpu CPU
Setup complete  (8 CPUs, 15.9 GB RAM, 366.4/464.4 GB disk)


In [5]:
path_to_annotated = "..\\data\\annotated_data\\train"
annotations = ("my_annotations", "all_annotations")

Create data configs for both annotation sets.

In [6]:
data_configs = []
for a in annotations:
    path_to_tiles = f"{path_to_annotated}\\{a}"
    config_data = {
        'path': os.path.abspath(path_to_tiles),
        'train': os.path.abspath(f"{path_to_tiles}\\train\\images"),
        'val': os.path.abspath(f"{path_to_tiles}\\val\\images"),
        'test':os.path.abspath("..\\data\\annotated_data\\test\\images"), # this is by default
        'names': {
            0: 'tree'
        }
    }
    config = f"{path_to_tiles}\\train_config.yaml"
    data_configs.append(config)
    with open(config, "w") as file:
        yaml.dump(config_data, file)

In [7]:
data_configs

['..\\data\\annotated_data\\train\\my_annotations\\train_config.yaml',
 '..\\data\\annotated_data\\train\\all_annotations\\train_config.yaml']

In [8]:
hyperamaters = {
    "data": data_configs,
    "model": ["yolov8n.pt", "yolov8m.pt", "yolov8x.pt"],
    "img_size": [250, 640, 1000, 1500],
}
model_settings = pd.DataFrame(
    [row for row in product(*hyperamaters.values())],
    columns=hyperamaters.keys()
)

model_settings["annotations"] = model_settings["data"].str.extract("(my_annotations|all_annotations)")
model_settings["project"] =  "models\\" + model_settings["annotations"]
model_settings["name"] = (
    model_settings["annotations"] + "_" +
    model_settings["model"] + "_" +
    model_settings["img_size"].astype(str)
)

In [9]:
# Test run
hyperamater_settings = model_settings.iloc[0, :]

model = ultralytics.YOLO(hyperamater_settings["model"])

model.train(
    data = hyperamater_settings["data"],
    epochs = 3,
    imgsz = int(hyperamater_settings["img_size"]),
    name = "test_run",
    project = ".test_runs"
)

Ultralytics YOLOv8.0.112  Python-3.11.3 torch-2.0.1+cpu CPU
yolo\engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=..\data\annotated_data\train\my_annotations\train_config.yaml, epochs=3, patience=50, batch=16, imgsz=250, save=True, save_period=-1, cache=False, device=None, workers=8, project=.test_runs, name=test_run, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, fraction=1.0, profile=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=torchscript, keras=False, optimize=False, int

In [ ]:
for i, settings in model_settings.iterrows():
    model = ultralytics.YOLO(settings["model"])
    model.train(
        data = settings["data"],
        epochs = 350,
        imgsz = int(settings["img_size"]),
        name = settings["name"],
        project = settings["project"]
    )